In [1]:
!pip install -q xlrd
!pip install -q openpyxl

In [2]:
import numpy as np
import pandas as pd
import re, string
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import fastprogress
from collections import Counter

In [3]:
df = pd.read_excel('../input/learning/input_data-1.xlsx')
df['Description'] = df['Description'].str.replace(r'(_x000D_\n)+', ' ', regex=True)
df['Description'] = df['Description'].str.replace('(employee# &', 'employee_num', regex=False)
df['combined desc'] = df['Short description'] + ' ' + df['Description']
df = df.dropna().reset_index(drop=True)

In [4]:
label_dist = df['Assignment group'].value_counts().reset_index()
drop_classes = label_dist[label_dist['Assignment group'] == 1]['index'].values
df = df[~df['Assignment group'].isin(drop_classes)].reset_index(drop=True)

In [5]:
for k, item in df.iterrows():
    df.loc[k, 'combined desc'] = item['combined desc'].replace('.'.join(item['Caller'].split()), 'ename')
    df.loc[k, 'combined desc'] = item['combined desc'].replace(item['Caller'].split()[0], 'first_name')
    df.loc[k, 'combined desc'] = item['combined desc'].replace(item['Caller'].split()[1], 'second_name')

In [6]:
def clean_numbers(x):
    return re.sub(r'\d+', ' ', x)

In [7]:
df['combined desc'] = df['combined desc'].str.lower()
df['combined desc'] = df['combined desc'].apply(clean_numbers)
df['combined desc'] = df['combined desc'].str.replace('(ticket_no)', 'ticket_no', regex=False)
df['combined desc'] = df['combined desc'].str.strip()

In [8]:
!python -m spacy download xx_ent_wiki_sm

     |████████████████████████████████| 3.7 MB 5.0 MB/s 
  Created wheel for xx-ent-wiki-sm: filename=xx_ent_wiki_sm-2.3.0-py3-none-any.whl size=3741307 sha256=df65326d570460bfc147624309e445079ab2ad8bb404e9e5fee64f95abf470e6
  Stored in directory: /root/.cache/pip/wheels/6d/3a/e2/6f3c30d9b864273b5c88e6cf82e83bdafff5296cd949915598
Successfully built xx-ent-wiki-sm
✔ Download and installation successful
You can now load the model via spacy.load('xx_ent_wiki_sm')


In [9]:
from spacy.tokenizer import Tokenizer
import xx_ent_wiki_sm
nlp = xx_ent_wiki_sm.load()

In [10]:
tokenizer = Tokenizer(nlp.vocab)
df['combined desc'] = df['combined desc'].map(tokenizer).map(lambda x: ' '.join(x.text.split()))

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

2021-09-15 15:51:22.634525: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [12]:
# cats = df['Assignment group'].unique().tolist()
# cat_dict = {j:i for i,j in enumerate(sorted(cats))}

In [13]:
x_train, x_val, y_train, y_val = train_test_split(df['combined desc'], df['Assignment group'].map(lambda x: x!='GRP_0').astype(float), random_state=2021, shuffle=True)

In [14]:
embed_size = 200
max_features = 10000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use
EMBEDDING_FILE='../input/glove6b/glove.6B.200d.txt'

In [15]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
list_tokenized_train = tokenizer.texts_to_sequences(x_train)
list_tokenized_test = tokenizer.texts_to_sequences(x_val)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [16]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [17]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


(-0.008671864, 0.38186216)

In [18]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [19]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])

2021-09-15 15:52:00.304608: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-15 15:52:00.307189: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-15 15:52:00.346251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 15:52:00.346870: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-15 15:52:00.346946: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-15 15:52:00.347003: I tensorflow/stream_executor/platform/def

In [20]:
model.fit(X_t, y_train, validation_data=(X_te, y_val), batch_size=32, epochs=8);

2021-09-15 15:52:03.051220: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-15 15:52:03.062258: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000179999 Hz


Epoch 1/8


2021-09-15 15:52:12.268599: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-15 15:52:13.094609: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


199/199 [==============================] - 319s 2s/step - loss: 0.5306 - binary_accuracy: 0.7198 - val_loss: 0.3480 - val_binary_accuracy: 0.8435
Epoch 2/8
199/199 [==============================] - 309s 2s/step - loss: 0.2894 - binary_accuracy: 0.8809 - val_loss: 0.3255 - val_binary_accuracy: 0.8520
Epoch 3/8
199/199 [==============================] - 312s 2s/step - loss: 0.2208 - binary_accuracy: 0.9105 - val_loss: 0.3707 - val_binary_accuracy: 0.8426
Epoch 4/8
199/199 [==============================] - 310s 2s/step - loss: 0.1553 - binary_accuracy: 0.9451 - val_loss: 0.3962 - val_binary_accuracy: 0.8478
Epoch 5/8
199/199 [==============================] - 313s 2s/step - loss: 0.1044 - binary_accuracy: 0.9634 - val_loss: 0.4357 - val_binary_accuracy: 0.8478
Epoch 6/8
199/199 [==============================] - 314s 2s/step - loss: 0.0789 - binary_accuracy: 0.9742 - val_loss: 0.5771 - val_binary_accuracy: 0.8421
Epoch 7/8
199/199 [==============================] - 315s 2s/step - loss: 